In [1]:
import glob
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from IPython.display import display


In [2]:
#UTILITY FUNCTIONS
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df



# BUILDING jobs_data.csv FILE (all data related to each launched job)

In [3]:
def read_jobs_data_folder(path, cnd):
    all_files = glob.glob(os.path.join(path , "*.txt"))
    li = []
    for filename in all_files:
        #print("Working on: "+filename)
        df = pd.read_csv(filename, index_col="jobid",header=None,sep=r"\s+", names=["jobid", "jobname","start_ts", "end_ts", "nodes"])
        li.append(df)
    
    df = pd.concat(li, axis=0)#, ignore_index=True)
    df['condition_n']=cnd
    print("Found "+str(len(df))+ " jobs with condition number "+str(cnd))
    return df

df1=read_jobs_data_folder(r'./jobs_data/cnd1/', 1)
df1000=read_jobs_data_folder(r'./jobs_data/cnd1000/', 1000)


df = pd.concat([df1,df1000], axis=0)

df.head()



Found 3612 jobs with condition number 1
Found 3623 jobs with condition number 1000


,jobname,start_ts,end_ts,nodes,condition_n
jobid,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_W15_20231130_023622,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:48*cresco6x183:48*cresco6x303:48*cresco6x056:48*cresco6x094:48*cresco6x163:48*cresco6x229:48*cresco6x157:48*cresco6x195:48*cresco6x017:48*cresco6x230,1
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:48*cresco6x163:48*cresco6x303:48*cresco6x229:48*cresco6x195:48*cresco6x114:48*cresco6x183:48*cresco6x017:48*cresco6x147:48*cresco6x051:48*cresco6x157:48*cresco6x094,1
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023622,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:48*cresco6x340:48*cresco6x023:48*cresco6x084:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x332:48*cresco6x134:48*cresco6x017:48*cresco6x051,1
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:48*cresco6x094:48*cresco6x147:48*cresco6x195:48*cresco6x157:48*cresco6x114:48*cresco6x214:48*cresco6x230:48*cresco6x056:48*cresco6x183:48*cresco6x051:48*cresco6x246,1
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023623,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x023:48*cresco6x221:48*cresco6x299:48*cresco6x340:48*cresco6x332:48*cresco6x084:48*cresco6x017:48*cresco6x303:48*cresco6x195,1


## INFERING jobs specifications from their names 

In [4]:

df['algorithm']=None
df['fault_tolerance']=None
df['fault_happened']=None
df['computing_proc']=None
df['total_proc']=None
df['matrix_size']=None
df['precision']=None
df['balanced']=None
df['repetition']=None
df['req_whole_time']=None
df['number_of_nodes']=None

for i in range(1,17):
    df['node'+str(i)]=0 
    

for index, row in df.iterrows():
    el=row['jobname'].split('_')
    df.at[index, 'algorithm']=el[0].split('FT')[0]
    df.at[index, 'fault_tolerance']=int(el[0].split('FT')[1])
    if df.at[index, 'fault_tolerance']!=0:
        df.at[index, 'fault_happened']= int(el[1].split('nf')[1])
        count_el=2
    else:
        df.at[index, 'fault_happened']= 0
        count_el=1
    df.at[index, 'computing_proc']=int(el[count_el].split('cp')[1])
    count_el+=1
    df.at[index, 'total_proc']=int(el[count_el].split('tnp')[1])
    count_el+=1
    df.at[index, 'matrix_size']=int(el[count_el].split('ms')[1])
    count_el+=1
    df.at[index, 'precision']=el[count_el] #single or double
    count_el+=1
    df.at[index, 'balanced']=el[count_el].split('lb')[1] #y or n
    count_el+=1
    df.at[index, 'repetition']=int(el[count_el].split('r')[1]) 
    count_el+=1
    df.at[index, 'req_whole_time']=int(el[count_el].split('W')[1]) 

    df.at[index, 'number_of_nodes']=row['nodes'].count('cresco')

    #i=1
    #for node in df.at[index, 'nodes'].split(':'):
    #    df.at[index,'node'+str(i)]=int(node.split('*')[0])
    #    i+=1


df.head()

,jobname,start_ts,end_ts,nodes,condition_n,algorithm,fault_tolerance,fault_happened,computing_proc,total_proc,matrix_size,precision,balanced,repetition,req_whole_time,number_of_nodes,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,node11,node12,node13,node14,node15,node16
jobid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_W15_20231130_023622,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:48*cresco6x183:48*cresco6x303:48*cresco6x056:48*cresco6x094:48*cresco6x163:48*cresco6x229:48*cresco6x157:48*cresco6x195:48*cresco6x017:48*cresco6x230,1,SPK,2,2,576,578,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:48*cresco6x163:48*cresco6x303:48*cresco6x229:48*cresco6x195:48*cresco6x114:48*cresco6x183:48*cresco6x017:48*cresco6x147:48*cresco6x051:48*cresco6x157:48*cresco6x094,1,IMeCO,4,0,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023622,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:48*cresco6x340:48*cresco6x023:48*cresco6x084:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x332:48*cresco6x134:48*cresco6x017:48*cresco6x051,1,SPK,4,0,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:48*cresco6x094:48*cresco6x147:48*cresco6x195:48*cresco6x157:48*cresco6x114:48*cresco6x214:48*cresco6x230:48*cresco6x056:48*cresco6x183:48*cresco6x051:48*cresco6x246,1,IMeCO,4,4,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023623,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x023:48*cresco6x221:48*cresco6x299:48*cresco6x340:48*cresco6x332:48*cresco6x084:48*cresco6x017:48*cresco6x303:48*cresco6x195,1,SPK,4,4,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## LOADING jobs specifications from jobs_spec folder

In [5]:
def read_jobs_spec_folder(path, cnd):
    all_files = glob.glob(os.path.join(path , "*.txt"))
    li = []
    for filename in all_files:
        print("Working on: "+filename)
        with open(filename, 'r') as file:
            file_contents = file.read()
            updated_contents = file_contents.replace("JOB NAME", "JOB_NAME")
        with open(filename, 'w') as file:
            file.write(updated_contents)
        df = pd.read_fwf(filename, colspecs="infer", infer_nrows=1000)
        li.append(df) 
    df = pd.concat(li, axis=0)#, ignore_index=True)
    df['condition_n_spec']=cnd
    print("Found "+str(len(df))+ " jobs with condition number "+str(cnd))
    return df

df1=read_jobs_spec_folder(r'./jobs_spec/cnd1/', 1)
df1000=read_jobs_spec_folder(r'./jobs_spec/cnd1000/', 1000)


df_spec = pd.concat([df1,df1000], axis=0)
df_spec = df_spec.set_index('JOB_NAME')
df_spec= df_spec.rename(columns={"FL": "IMe_fault_level", "FR": "IMe_fault_rank", "SPK-CP": "SPK_checkpoint_iteration" , "BKF": "blocking_factor"  })

df_spec.head()

Working on: ./jobs_spec/cnd1/20231129_150132_report.txt
Working on: ./jobs_spec/cnd1/20231228_231746_report.txt
Working on: ./jobs_spec/cnd1/20231228_101745_report.txt
Working on: ./jobs_spec/cnd1/20231123_005007_report.txt
Working on: ./jobs_spec/cnd1/20231229_014910_report.txt
Working on: ./jobs_spec/cnd1/20231228_101446_report.txt
Working on: ./jobs_spec/cnd1/20231126_082959_report.txt
Working on: ./jobs_spec/cnd1/20240102_150154_report.txt
Working on: ./jobs_spec/cnd1/20231130_023556_report.txt
Working on: ./jobs_spec/cnd1/20231205_163342_report.txt
Working on: ./jobs_spec/cnd1/20231125_072327_report.txt
Working on: ./jobs_spec/cnd1/20231228_203243_report.txt
Working on: ./jobs_spec/cnd1/20231125_110031_report.txt
Working on: ./jobs_spec/cnd1/20231121_222647_report.txt
Working on: ./jobs_spec/cnd1/20231125_072604_report.txt
Working on: ./jobs_spec/cnd1/20231226_165106_report.txt
Working on: ./jobs_spec/cnd1/20231228_231413_report.txt
Working on: ./jobs_spec/cnd1/20231228_101831_rep

,ALGORITHM,PRECISION,FT,NF,CPROCS,SPROCS,TPROCS,MATSIZE,NODES,IMe_fault_level,IMe_fault_rank,SPK_checkpoint_iteration,blocking_factor,SBAL,NxSOCK,R,WT,condition_n_spec
JOB_NAME,,,,,,,,,,,,,,,,,,
IMeCOFT0_cp400_tnp400_ms36960_single_lbn_r1_W10_20231129_150138,IMeCO,single,-,-,400,0,400,36960,9,-,-,-,-,N,-,1,10,1
SPKFT0_cp400_tnp400_ms36960_single_lbn_r1_W10_20231129_150138,SPK,single,-,-,400,0,400,36960,9,-,-,-,-,N,-,1,10,1
IMeCOFT1_nf0_cp400_tnp420_ms36960_single_lbn_r1_W10_20231129_150138,IMeCOft,single,1,0,400,20,420,36960,9,2,2,-,-,N,-,1,10,1
SPKFT1_nf0_cp400_tnp401_ms36960_single_lbn_r1_W10_20231129_150139,SPKft,single,1,0,400,1,401,36960,9,2,2,18480,24,N,-,1,10,1
IMeCOFT1_nf1_cp400_tnp420_ms36960_single_lbn_r1_W10_20231129_150139,IMeCOft,single,1,1,400,20,420,36960,9,2,2,-,-,N,-,1,10,1


## JOINING information in jobs_data with jobs_spec and checking consistency

In [6]:
df_join=df.reset_index().merge(df_spec, left_on='jobname',right_on='JOB_NAME', how='left').set_index('jobid')
#temp=df_join[df_join['ALGORITHM'].isnull()]

check=0
for index, row in df_join.iterrows():
    if  not pd.isna(row['ALGORITHM']) :
        if row['algorithm'] != str(row['ALGORITHM']).replace("ft",""):
            print("jobname="+str(row['jobname'])+" diff in  row['ALGORITHM']="+str(row['ALGORITHM'])+" row['algorithm']="+str(row['algorithm']))
            check=1
        if  row['precision']!=row['PRECISION']:
            print("jobname="+str(row['jobname'])+" diff in  row['precision']="+str(row['precision'])+" row['PRECISION']="+str(row['PRECISION']))
            check=1
        if  (str(row['fault_tolerance'])!='0' and str(row['fault_tolerance'])!=str(row['FT'])) or (str(row['fault_tolerance'])=='0' and str(row['FT'])!='-') :
            print("jobname="+str(row['jobname'])+" diff in  row['fault_tolerance']="+str(row['fault_tolerance'])+" row['FT']="+str(row['FT']))
            check=1
        if  (str(row['fault_happened'])!='0' and str(row['fault_happened'])!=str(row['NF']))      or    (str(row['fault_happened'])=='0' and str(row['NF']) not in ['0','-'] ) :
            print("jobname="+str(row['jobname'])+" diff in  row['fault_happened']="+str(row['fault_happened'])+" row['NF']="+str(row['NF']))
            check=1
        if  row['computing_proc']!=int(row['CPROCS']):
            print("jobname="+str(row['jobname'])+" diff in  row['computing_proc']="+str(row['computing_proc'])+" row['CPROCS']="+str(row['CPROCS']))
            check=1
        if  row['total_proc']!=int(row['TPROCS']):
            print("jobname="+str(row['jobname'])+" diff in  row['total_proc']="+str(row['total_proc'])+" row['TPROCS']="+str(row['TPROCS']))
            check=1
        if  row['matrix_size']!=int(row['MATSIZE']):
            print("jobname="+str(row['jobname'])+" diff in  row['matrix_size']="+str(row['matrix_size'])+" row['MATSIZE']="+str(row['MATSIZE']))
            check=1
        if  row['number_of_nodes']!=int(row['NODES']):
            print("jobname="+str(row['jobname'])+" diff in  row['number_of_nodes']="+str(row['number_of_nodes'])+" row['NODES']="+str(row['NODES']))
            check=1
        if  row['balanced'].upper()!=row['SBAL']:
            print("jobname="+str(row['jobname'])+" diff in  row['balanced']="+str(row['balanced'])+" row['SBAL']="+str(row['SBAL']))
            check=1
        if  row['req_whole_time']!=int(row['WT']):
            print("jobname="+str(row['jobname'])+" diff in  row['req_whole_time']="+str(row['req_whole_time'])+" row['WT']="+str(row['WT']))
            check=1
        if  row['condition_n']!=int(row['condition_n_spec']):
            print("jobname="+str(row['jobname'])+" diff in  row['condition_n']="+str(row['condition_n'])+" row['condition_n_spec']="+str(row['condition_n_spec']))
            check=1
if (check==0):
    print("All jobnames are consistent with the job's specifications. Dropping redundant columns...")
    df_join=df_join.drop(['ALGORITHM', 'PRECISION', 'FT', 'NF', 'CPROCS', 'TPROCS','SPROCS', 'MATSIZE', 'NODES', 'SBAL', 'WT', 'condition_n_spec'], axis=1)
    print('Using NxSOCK to compute the number of processors used on each node [i.e., columns node1...node16]')
    #count=0
    for index, row in df_join.iterrows():
        if (row['balanced']=='y') and not pd.isna(row['NxSOCK']) :
            for n in range(row['number_of_nodes']):
                col_name='node'+str(n+1)
                df_join.at[index, col_name] = int(row['NxSOCK'])*2
            df_join.at[index, col_name] = int(row['NxSOCK'])*2 - (int(row['NxSOCK'])*2*row['number_of_nodes'] - row['total_proc'])
                  
df_join = df_join[df_join['NxSOCK'].notna()]

All jobnames are consistent with the job's specifications. Dropping redundant columns...
Using NxSOCK to compute the number of processors used on each node [i.e., columns node1...node16]


# Creating a colum to report termination with/without error

In [7]:
df=df_join
#creating a colum to report termination with/without error
df['error_code']=0

for index, row in df.iterrows():
    foundErr=False
    for dirpath, dirnames, filenames in os.walk("./results/"):
        for filename in [f for f in filenames if f==(row['jobname']+".err")]:
            errorfile=os.path.join(dirpath, filename)
            csvfile=errorfile.replace("err","csv")
            foundErr=True
            break
        if foundErr==True:
            break

    if not os.path.exists(errorfile): #err file does not exist
        print("jobid="+str(index)+": No .err file for jobname="+row['jobname']+". Setting to 1.")
        df.at[index, 'error_code'] = 1
    elif os.path.getsize(errorfile)!=0: #err file exist and has content
        print("jobid="+str(index)+": Errorfile has content for jobname="+row['jobname'])
        df.at[index, 'error_code'] = 1

    if not os.path.exists(csvfile):
        print("jobid="+str(index)+": No .csv file for jobname="+row['jobname']+".")
        if df.at[index, 'error_code']==0:
            print("error_code was 0. Setting to 1.")
            df.at[index, 'error_code'] = 1
    else: #csv file exist
        with open(csvfile, "r") as f:
            for line in f:
                pass
            try:
                csv_error_code = int(line.split(',')[3])
            except IndexError:
                print("jobid="+str(index)+": No standard format for csvfile="+csvfile) 
                continue
            if csv_error_code != 0 :
                #print("Last line is "+line)
                #print("jobid="+str(index)+": Csvfile reports csv_error_code="+str(csv_error_code)+" for jobname="+row['jobname']+". Overriding error_code="+str(df.at[index, 'error_code']))
                df.at[index, 'error_code'] = csv_error_code
         
    
df[df['error_code'] != 0].head()


jobid=891582: Errorfile has content for jobname=SPKFT8_nf0_cp576_tnp584_ms42240_double_lbn_r1_W15_20231130_023623
jobid=881812: Errorfile has content for jobname=IMeCOFT2_nf2_cp484_tnp528_ms15840_double_lbn_r1_W3_20231123_005128
jobid=881812: No .csv file for jobname=IMeCOFT2_nf2_cp484_tnp528_ms15840_double_lbn_r1_W3_20231123_005128.
jobid=937422: Errorfile has content for jobname=IMeCOFT4_nf4_cp256_tnp320_ms42240_single_lby_r1_W40_20231226_165115
jobid=887340: Errorfile has content for jobname=SPKFT1_nf1_cp484_tnp485_ms26400_double_lbn_r1_W7_20231126_083057
jobid=881573: Errorfile has content for jobname=SPKFT1_nf0_cp400_tnp401_ms15840_single_lbn_r1_W3_20231123_005057
jobid=880090: Errorfile has content for jobname=IMeCOFT1_nf0_cp144_tnp156_ms26400_double_lbn_r1_W20_20231121_222707
jobid=890918: Errorfile has content for jobname=IMeCOFT4_nf4_cp400_tnp480_ms36960_double_lby_r1_W10_20231129_150148
jobid=890920: Errorfile has content for jobname=IMeCOFT8_nf0_cp400_tnp560_ms36960_double_l

,jobname,start_ts,end_ts,nodes,condition_n,algorithm,fault_tolerance,fault_happened,computing_proc,total_proc,matrix_size,precision,balanced,repetition,req_whole_time,number_of_nodes,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,node11,node12,node13,node14,node15,node16,IMe_fault_level,IMe_fault_rank,SPK_checkpoint_iteration,blocking_factor,NxSOCK,R,error_code
jobid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_W15_20231130_023622,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:48*cresco6x183:48*cresco6x303:48*cresco6x056:48*cresco6x094:48*cresco6x163:48*cresco6x229:48*cresco6x157:48*cresco6x195:48*cresco6x017:48*cresco6x230,1,SPK,2,2,576,578,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023623,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x023:48*cresco6x221:48*cresco6x299:48*cresco6x340:48*cresco6x332:48*cresco6x084:48*cresco6x017:48*cresco6x303:48*cresco6x195,1,SPK,4,4,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99
891582,SPKFT8_nf0_cp576_tnp584_ms42240_double_lbn_r1_W15_20231130_023623,1701591716,1701591788,48*cresco6x018:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x218:48*cresco6x044:48*cresco6x225:48*cresco6x202:48*cresco6x301:48*cresco6x303:48*cresco6x321:48*cresco6x023:48*cresco6x188,1,SPK,8,0,576,584,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,1
891584,SPKFT8_nf8_cp576_tnp584_ms42240_double_lbn_r1_W15_20231130_023623,1701592504,1701592573,48*cresco6x206:48*cresco6x225:48*cresco6x074:48*cresco6x018:48*cresco6x303:48*cresco6x188:48*cresco6x218:48*cresco6x321:48*cresco6x310:48*cresco6x202:48*cresco6x301:48*cresco6x023:48*cresco6x062,1,SPK,8,8,576,584,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99
891590,SPKFT1_nf1_cp576_tnp577_ms42240_single_lby_r1_W15_20231130_023624,1701593721,1701593767,48*cresco6x023:48*cresco6x246:48*cresco6x218:48*cresco6x147:48*cresco6x176:48*cresco6x095:48*cresco6x321:48*cresco6x332:48*cresco6x340:48*cresco6x074:48*cresco6x134:48*cresco6x303:48*cresco6x225,1,SPK,1,1,576,577,42240,single,y,1,15,13,46,46,46,46,46,46,46,46,46,46,46,46,25,0,0,0,2,2,21120,22,23,1.0,-99


# Creating colums to report the run times (whole time by LSF--including process dipatching and termination, whole time--including inizialization of the algorithm, whole time--without initialization)

In [8]:
#creating a colum to report the run time (whole time by LSF, including process dipatching and termination)
df['runtime']=0
#creating a colum to report the run time of the routine call (whole time, including inizialization of the algorithm)
df['init_and_call_runtime']=0
#creating a colum to report the pure run time of the routine call (no inizialization)
df['call_runtime']=0


for index, row in df.iterrows():
    txtfile_found=False
    for dirpath, dirnames, filenames in os.walk("./results/"):
        for filename in [f for f in filenames if f==(row['jobname']+".txt")]:
            txtfile=os.path.join(dirpath, filename)
            txtfile_found=True
            #print("Examining "+row['jobname']+".txt ...")
            break
        if txtfile_found==True:
            break
   
    if txtfile_found==True: 
        with open(txtfile, "r") as searchfile:
            runtime_found=False
            call_runtime_found=False
            for line in searchfile:
                left,sep,right = line.partition('Run time :')
                if sep: 
                    #print("jobid="+str(index)+" jobname="+row['jobname']+" runtime is "+right.split('sec.')[0].strip())
                    df.at[index, 'runtime']=int(right.split('sec.')[0].strip())
                    runtime_found=True
                left,sep,right = line.partition('Call    run time:')
                if sep:
                    both=right.split('s')[0].strip().split()
                    df.at[index, 'init_and_call_runtime']=int(both[0])
                    df.at[index, 'call_runtime']=int(both[1].replace('(','').replace(')',''))
                    #print("jobid="+str(index)+" init_and_call_runtime is "+row['init_and_call_runtime']+" call_runtime is "+row['call_runtime'])
                    call_runtime_found=True
                    
            if runtime_found==False and row['error_code']==0:
                    print("jobid="+str(index)+": No 'Run time :' string in file: "+txtfile+" row['error']="+str(row['error']))
            if call_runtime_found==False and row['error_code']==0:
                print("jobid="+str(index)+": No 'Call    run time:' string in file: "+txtfile+" row['error']="+str(row['error']))
    
    if txtfile_found==False:
        print("jobid="+str(index)+": No .txt file for jobid="+str(index)+" jobname="+row['jobname'])
        #row['error']=1
df.head()


,jobname,start_ts,end_ts,nodes,condition_n,algorithm,fault_tolerance,fault_happened,computing_proc,total_proc,matrix_size,precision,balanced,repetition,req_whole_time,number_of_nodes,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,node11,node12,node13,node14,node15,node16,IMe_fault_level,IMe_fault_rank,SPK_checkpoint_iteration,blocking_factor,NxSOCK,R,error_code,runtime,init_and_call_runtime,call_runtime
jobid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_W15_20231130_023622,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:48*cresco6x183:48*cresco6x303:48*cresco6x056:48*cresco6x094:48*cresco6x163:48*cresco6x229:48*cresco6x157:48*cresco6x195:48*cresco6x017:48*cresco6x230,1,SPK,2,2,576,578,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99,71,47,32
891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:48*cresco6x163:48*cresco6x303:48*cresco6x229:48*cresco6x195:48*cresco6x114:48*cresco6x183:48*cresco6x017:48*cresco6x147:48*cresco6x051:48*cresco6x157:48*cresco6x094,1,IMeCO,4,0,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,-,-,-,1.0,0,685,659,652
891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023622,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:48*cresco6x340:48*cresco6x023:48*cresco6x084:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x332:48*cresco6x134:48*cresco6x017:48*cresco6x051,1,SPK,4,0,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,0,69,47,32
891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r1_W15_20231130_023622,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:48*cresco6x094:48*cresco6x147:48*cresco6x195:48*cresco6x157:48*cresco6x114:48*cresco6x214:48*cresco6x230:48*cresco6x056:48*cresco6x183:48*cresco6x051:48*cresco6x246,1,IMeCO,4,4,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,-,-,-,1.0,0,612,588,581
891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_W15_20231130_023623,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:48*cresco6x206:48*cresco6x023:48*cresco6x221:48*cresco6x299:48*cresco6x340:48*cresco6x332:48*cresco6x084:48*cresco6x017:48*cresco6x303:48*cresco6x195,1,SPK,4,4,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99,72,47,32


### OPTIONAL CHECK: difference between RUNTIME and end_ts - start_ts:

In [9]:
for index, row in df.iterrows():
    diff = row['end_ts'] - row['start_ts']
    if abs(diff - row['runtime'])>1 and row['error_code']==0:
        print("jobid="+str(index)+" jobname="+row['jobname']+" diff="+str(diff)+" row['runtime']="+str(row['runtime']))

jobid=884392 jobname=IMeCOFT2_nf0_cp100_tnp120_ms5280_single_lbn_r1_W5_20231125_072341 diff=8 row['runtime']=16
jobid=884600 jobname=IMeCOFT8_nf8_cp64_tnp128_ms10560_double_lby_r1_W8_20231125_072617 diff=62 row['runtime']=71
jobid=884845 jobname=IMeCOFT2_nf2_cp144_tnp168_ms10560_double_lbn_r1_W8_20231125_072650 diff=36 row['runtime']=44
jobid=884848 jobname=SPKFT4_nf0_cp144_tnp148_ms10560_double_lbn_r1_W8_20231125_072650 diff=12 row['runtime']=23
jobid=884852 jobname=SPKFT8_nf0_cp144_tnp152_ms10560_double_lbn_r1_W8_20231125_072651 diff=12 row['runtime']=20
jobid=884856 jobname=SPKFT0_cp144_tnp144_ms10560_single_lby_r1_W8_20231125_072651 diff=8 row['runtime']=17
jobid=942487 jobname=IMeCOFT1_nf1_cp144_tnp156_ms31680_double_lbn_r1_W60_20240102_150203 diff=874 row['runtime']=885
jobid=942499 jobname=IMeCOFT0_cp144_tnp144_ms36960_double_lby_r1_W60_20240102_150204 diff=1380 row['runtime']=1390
jobid=889545 jobname=IMeCOFT8_nf8_cp484_tnp660_ms31680_double_lby_r1_W8_20231128_101631 diff=288 r

In [10]:
df.to_csv('jobs_data.csv', float_format='%f',index=True)
print("job_data.csv DONE!")


job_data.csv DONE!
